# BERT for classification

## Import the dataset

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


## Bert + softmax

In [ ]:
import torch
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from datasets import load_dataset
from tqdm import tqdm  # For progress bars

class SentimentClassifier:
    def __init__(self, model_name='bert-base-uncased', num_labels=5, lr=2e-5, batch_size=16):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.optimizer = AdamW(self.model.parameters(), lr=lr, weight_decay=0.1)
        self.batch_size = batch_size

        # Check if GPU is available
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)  # Move model to GPU if available

    def load_data(self):
        # Load the SetFit/sst5 dataset
        dataset = load_dataset("SetFit/sst5")
        train_dataset = dataset['train']
        dev_dataset = dataset['test']
        return train_dataset, dev_dataset

    def preprocess_data(self, sentences, labels):
        inputs = self.tokenizer(sentences, return_tensors='pt', max_length=512, truncation=True, padding=True)
        labels = torch.tensor(labels)
        return inputs, labels

    def train(self, epochs=3):
        # Load and preprocess the dataset
        train_dataset, dev_dataset = self.load_data()
        train_inputs, train_labels = self.preprocess_data(train_dataset['text'], train_dataset['label'])

        # Move data to GPU if available
        train_inputs = {key: val.to(self.device) for key, val in train_inputs.items()}
        train_labels = train_labels.to(self.device)

        total_training_time = 0

        for epoch in range(epochs):
            start_time = time.time()
            self.model.train()
            total_loss = 0

            # Progress bar for batches
            progress_bar = tqdm(range(0, len(train_labels), self.batch_size), desc=f"Epoch {epoch + 1}/{epochs}")

            for i in progress_bar:
                b_input_ids = train_inputs['input_ids'][i:i+self.batch_size]
                b_attention_mask = train_inputs['attention_mask'][i:i+self.batch_size]
                b_labels = train_labels[i:i+self.batch_size]

                self.model.zero_grad()
                outputs = self.model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
                loss = outputs.loss
                total_loss += loss.item()
                loss.backward()
                self.optimizer.step()

                # Update progress bar with the current loss
                progress_bar.set_postfix(loss=loss.item())

            avg_train_loss = total_loss / (len(train_labels) // self.batch_size)
            epoch_time = time.time() - start_time
            total_training_time += epoch_time
            print(f"Epoch {epoch + 1} completed in {epoch_time:.2f} seconds. Loss: {avg_train_loss:.4f}")

            # Estimate remaining time
            remaining_time = (epochs - (epoch + 1)) * (total_training_time / (epoch + 1))
            print(f"Estimated remaining time: {remaining_time:.2f} seconds")

            # Evaluation on dev set
            avg_dev_accuracy = self.evaluate(dev_dataset['text'], dev_dataset['label'])
            print(f"Epoch {epoch + 1}, Dev Accuracy: {avg_dev_accuracy:.4f}")

        print(f"Total training time: {total_training_time:.2f} seconds")

    def evaluate(self, sentences, labels):
        self.model.eval()
        inputs, labels = self.preprocess_data(sentences, labels)

        # Move data to GPU if available
        inputs = {key: val.to(self.device) for key, val in inputs.items()}
        labels = labels.to(self.device)

        with torch.no_grad():
            outputs = self.model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        logits = outputs.logits

        preds = torch.argmax(logits, dim=1)
        accuracy = (preds == labels).float().mean().item()

        return accuracy

In [ ]:
def train_and_test(params):
  dataset = load_dataset("SetFit/sst5")
  test_dataset = dataset['test']
  accuracies = dict()
  for param in params:
    classifier = SentimentClassifier(**param)
    classifier.train(epochs=3)
    accuracies[str(param)] = classifier.evaluate(test_dataset['text'], test_dataset['label'])
  return accuracies

In [ ]:
params = [
    {
        'lr': 1e-4,
        'batch_size': 16,
    },
    {
        'lr': 1e-5,
        'batch_size': 16,
    },
    {
        'lr': 1e-6,
        'batch_size': 16,
    },
    {
        'lr': 1e-4,
        'batch_size': 32,
    },
    {
        'lr': 1e-5,
        'batch_size': 32,
    },
    {
        'lr': 1e-6,
        'batch_size': 32,
    },
]

In [ ]:
accs = train_and_test(params)
print(accs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

...

Epoch 1 completed in 108.25 seconds. Loss: 1.5889
Estimated remaining time: 216.51 seconds
Epoch 1, Dev Accuracy: 0.3018


Epoch 2/3: 100%|██████████| 267/267 [01:48<00:00,  2.47it/s, loss=1.44]


Epoch 2 completed in 108.14 seconds. Loss: 1.4854
Estimated remaining time: 108.20 seconds
Epoch 2, Dev Accuracy: 0.4371


Epoch 3/3: 100%|██████████| 267/267 [01:48<00:00,  2.47it/s, loss=1.36]


Epoch 3 completed in 108.09 seconds. Loss: 1.3589
Estimated remaining time: 0.00 seconds
Epoch 3, Dev Accuracy: 0.4697
Total training time: 324.49 seconds
{"{'lr': 0.0001, 'batch_size': 16}": 0.4158371090888977, "{'lr': 1e-05, 'batch_size': 16}": 0.5253393650054932, "{'lr': 1e-06, 'batch_size': 16}": 0.459276020526886, "{'lr': 0.0001, 'batch_size': 32}": 0.5194570422172546, "{'lr': 1e-05, 'batch_size': 32}": 0.5266968607902527, "{'lr': 1e-06, 'batch_size': 32}": 0.4696832597255707}


# RoBERTa for classification

In [ ]:
params = list(map(lambda x: {**x, 'model_name': 'roberta-base'}, params))

accs_roberta = train_and_test(params)
print(accs_roberta)

Repo card metadata block was not found. Setting CardData to empty.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Epoch 1 completed in 107.73 seconds. Loss: 1.5915
Estimated remaining time: 215.46 seconds
Epoch 1, Dev Accuracy: 0.3575


Epoch 2/3: 100%|██████████| 267/267 [01:47<00:00,  2.48it/s, loss=1.19]


Epoch 2 completed in 107.82 seconds. Loss: 1.4257
Estimated remaining time: 107.77 seconds
Epoch 2, Dev Accuracy: 0.5014


Epoch 3/3: 100%|██████████| 267/267 [01:47<00:00,  2.48it/s, loss=1.07]


Epoch 3 completed in 107.82 seconds. Loss: 1.1485
Estimated remaining time: 0.00 seconds
Epoch 3, Dev Accuracy: 0.5299
Total training time: 323.37 seconds
{"{'lr': 0.0001, 'batch_size': 16, 'model_name': 'roberta-base'}": 0.23076923191547394, "{'lr': 1e-05, 'batch_size': 16, 'model_name': 'roberta-base'}": 0.5674208402633667, "{'lr': 1e-06, 'batch_size': 16, 'model_name': 'roberta-base'}": 0.5457013845443726, "{'lr': 0.0001, 'batch_size': 32, 'model_name': 'roberta-base'}": 0.5176470875740051, "{'lr': 1e-05, 'batch_size': 32, 'model_name': 'roberta-base'}": 0.5733031630516052, "{'lr': 1e-06, 'batch_size': 32, 'model_name': 'roberta-base'}": 0.5298642516136169}


# DistilBert for classification

In [ ]:
params = list(map(lambda x: {**x, 'model_name': 'distilbert-base-uncased'}, params))

accs_distilbert = train_and_test(params)
print(accs_distilbert)

Repo card metadata block was not found. Setting CardData to empty.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 1 completed in 54.92 seconds. Loss: 1.5756
Estimated remaining time: 109.83 seconds
Epoch 1, Dev Accuracy: 0.3593


Epoch 2/3: 100%|██████████| 267/267 [00:54<00:00,  4.88it/s, loss=1.47]


Epoch 2 completed in 54.69 seconds. Loss: 1.5153
Estimated remaining time: 54.81 seconds
Epoch 2, Dev Accuracy: 0.4249


Epoch 3/3: 100%|██████████| 267/267 [00:54<00:00,  4.86it/s, loss=1.39]


Epoch 3 completed in 54.96 seconds. Loss: 1.3692
Estimated remaining time: 0.00 seconds
Epoch 3, Dev Accuracy: 0.4552
Total training time: 164.57 seconds
{"{'lr': 0.0001, 'batch_size': 16, 'model_name': 'distilbert-base-uncased'}": 0.4429864287376404, "{'lr': 1e-05, 'batch_size': 16, 'model_name': 'distilbert-base-uncased'}": 0.5203620195388794, "{'lr': 1e-06, 'batch_size': 16, 'model_name': 'distilbert-base-uncased'}": 0.4791855216026306, "{'lr': 0.0001, 'batch_size': 32, 'model_name': 'distilbert-base-uncased'}": 0.4837104082107544, "{'lr': 1e-05, 'batch_size': 32, 'model_name': 'distilbert-base-uncased'}": 0.5140271782875061, "{'lr': 1e-06, 'batch_size': 32, 'model_name': 'distilbert-base-uncased'}": 0.4552036225795746}
